# 1、输出解析器

- 语言模型返回的内容通常都是字符串的格式（文本格式）、消息类型，但在实际AI应用开发过程中，往往希望大模型可以返回更直观、更格式化的内容，以确保应用能够顺利进行后续的逻辑处理
- 输出解析器（Output Parser）负责获取 LLM 的输出并将其转换为更合适的格式

## 1.1、输出解析器的分类

LangChain有许多不同类型的输出解析器
- StrOutputParser ：字符串解析器
- JsonOutputParser ：JSON解析器，确保输出符合特定JSON对象格式
- XMLOutputParser ：XML解析器，允许以流行的XML格式从LLM获取结果
- CommaSeparatedListOutputParser ：CSV解析器，模型的输出以逗号分隔，以列表形式返回输出
- DatetimeOutputParser ：日期时间解析器，可用于将 LLM 输出解析为日期时间格式
除了上述常用的输出解析器之外，还有：
- EnumOutputParser ：枚举解析器，将LLM的输出，解析为预定义的枚举值
- StructuredOutputParser ：将非结构化文本转换为预定义格式的结构化数据（如字典）
- OutputFixingParser ：输出修复解析器，用于自动修复格式错误的解析器，比如将返回的不符合
预期格式的输出，尝试修正为正确的结构化数据（如 JSON）
- RetryOutputParser ：重试解析器，当主解析器（如 JSONOutputParser）因格式错误无法解析
- LLM 的输出时，通过调用另一个 LLM 自动修正错误，并重新尝试解析

## 1.2、字符串解析器(StrOutputParser)

- StrOutputParser: 将任何输入转换为字符串,用于从回答中提取content字段
如下,将一个对话模型的输出结果，解析为字符串输出

In [4]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.messages import SystemMessage,HumanMessage
import os, dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
chat_model = ChatOpenAI(model="gpt-4o-mini")


messages = [
    SystemMessage(content="请将用户输入的中文翻译成英文"),
    HumanMessage(content="长风破浪会有时,直挂云帆济沧海")
]

response = chat_model.invoke(messages)
print(type(response))   # AIMessage
print(response)
# 如何获取一个字符串类型的输出结果
# 方式1: 调用回答的content
print(f"content : {response.content}")

# 方式2: 使用输出解析器得到回答的content
parser = StrOutputParser()
str_response = parser.invoke(response)
print(type(str_response))  # <class 'str'>
print(str_response)

<class 'langchain_core.messages.ai.AIMessage'>
content='When the wind is strong and the waves are high, I will set my sail to cross the vast ocean.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 38, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_efad92c60b', 'id': 'chatcmpl-CYqHkVlRLVOip6tOs3nJObF7Bk8hw', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--3c44cb35-81cb-48a4-8740-4c19e0575ceb-0' usage_metadata={'input_tokens': 38, 'output_tokens': 23, 'total_tokens': 61, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content : When the wind is strong and the waves are high, I will set my s

## 1.3、JSON解析器(JsonOutputParser)

- JsonOutputParser: 将大模型输出的 自由文本 转化为 结构化的JSON数据, 适用于需要严格结构化输出的场景, 比如: API调用,数据存储 或 下游任务处理
- 实现方式:
    - 方式1: 用户通过提示词指明返回JSON
    - 方式2: 借助JsonOutputParser的get_format_instructions(),生成格式说明，指导模型输出JSON 结构

In [3]:
# 导入必要模块
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

chat_model = ChatOpenAI(
    model = "deepseek-chat",
    base_url = os.environ["BASE_URL"],
    api_key = os.environ["DEEPSEEK_API_KEY"]
)

# 1. 定义JSON结构（用Pydantic模型指定输出格式）
class UserInfo(BaseModel):
    name: str = Field(description="用户姓名")
    age: int = Field(description="用户年龄")
    hobby: str = Field(description="用户爱好")

# 2. 创建JSON解析器
parser = JsonOutputParser(pydantic_object=UserInfo)

# 3. 格式说明（告知模型必须按此格式输出）
format_instructions = parser.get_format_instructions()

# 4. 构建提示词（包含格式要求）
prompt = PromptTemplate(
    template="请解析这句话：'我叫小明，25岁，喜欢打篮球'\n{format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": format_instructions}
)

# 5. 调用大模型
response = chat_model.invoke(prompt.format())

# 6. 用解析器处理输出
parsed_data = parser.parse(response.content)

# 展示结果
print("原始模型输出：")
print(response.content)
print("\n解析后的结构化数据：")
print(parsed_data)

原始模型输出：
{"name": "小明", "age": 25, "hobby": "打篮球"}

解析后的结构化数据：
{'name': '小明', 'age': 25, 'hobby': '打篮球'}


- 解析器有些鸡肋,一般都推荐告知大模型最后要输出什么格式的回答就好